# Overview

Definition of state monad

```
state -> (state, result)
```

- **A state monad is a function**, rather than a data structure
- **Given state**, state monad returns the **next state, and a result**
- Functions that **returns a state monad returns a function**

Compared to explicitly passing state to functions, **state monad allows state to be implicitly passed to functions, avoiding human errors**

In [ ]:
/// Normally the output of the state function is a tuple. 
/// However, for readibility, we will use a record instead
type StateAndAction<'state, 'result> = { State: 'state; Result: 'result }

/// State monad are typically called just the `State`. 
/// We call it `StateMonad` here for clarity
type StateMonad<'state, 'result> = 'state -> StateAndAction<'state, 'result>

let nextInt: StateMonad<int, int> = 
    fun i -> { State = i + 1; Result = i }
let nextBool: StateMonad<int, bool> = 
    fun i -> { State = (i + 1); Result = (i % 2 = 0) }

printfn $"{nextInt 0}"
printfn $"{nextBool 1}"

The below script needs to be able to find the current output cell; this is an easy method to get it.

{ State = 1
  Result = 0 }
{ State = 2
  Result = false }


# State Monad Operations

- `get`: returns an state monad that treats the `state` as the `result`
- `put newState`: returns a state monad that accept `newState` as its state
- `bind f monad`: returns a state monad that 
  1. Evalaute `monad` with the state, 
  2. Use `f` to create a monad from the result of step 1
  3. Evaluate the new monad using the state from step 1

## Helper Functions

- `evaluate state monad`: evaluate the `monad` using `state` and then returns the result

In [ ]:
module StateMonad = 
    let get: StateMonad<'state, 'state> = 
        (fun state -> { State = state; Result = state })
        
    let put (newState: 'state): StateMonad<'state, unit> = 
        (fun state -> { State = newState; Result = () })
    
    let evaluate (state: 'state) (monad: StateMonad<'state, 'result>): 'result =
        (monad state).Result
        
    let bind 
        (f: 'result -> StateMonad<'state, 'result2>) 
        (monad: StateMonad<'state, 'result>) 
        : StateMonad<'state, 'result2> = 
        (fun state -> 
            let snapshot = monad state
            let fMonad = f snapshot.Result
            
            fMonad snapshot.State)

type StateMonadBuilder() =    
    member this.Bind (monad, f) = StateMonad.bind f monad
    member this.Return (result: 'result): StateMonad<'state, 'result> = 
        (fun state -> { State = state; Result = result })
    
let state = new StateMonadBuilder()
state

FSI_0010+StateMonadBuilder

In [ ]:
let increment: StateMonad<int, int> = 
    state {
        let! counter = StateMonad.get
        do!
            StateMonad.put (counter + 1)
        return counter
    } 
    
let testIncrement = 
    state {
        let! result1 = increment 
        let! result2 = increment
        
        return (result1, result2)
    }
    
StateMonad.evaluate 0 testIncrement

Item1,Item2
0,1


- TODO: Indexed state monad
  - Enable state transition
- TODO: Indexed state transformer
- TODO: HList, Shapeless

# References

- [Regaining Control with State Monad and Friends (Felix Mulder)](https://www.youtube.com/watch?v=Pgo73GfHk0U)
- [jwosty/StateBuilder.fsx](https://gist.github.com/jwosty/5338fce8a6691bbd9f6f)